## CrypTen - Private Model Inference using Plans


We will do a simple inference on an encrypted neural network that is not known by the local worker.
The workers that know the model structure are deployed as [Grid Nodes](https://github.com/OpenMined/PyGrid/tree/dev/apps/node). For this we will be using Plans and we will be using CrypTen as a backend for SMPC. 


Authors:
 - George Muraru - Twitter: [@gmuraru](https://twitter.com/georgemuraru)
 - Ayoub Benaissa - Twitter: [@y0uben11](https://twitter.com/y0uben11)

## Inference Overivew
* In this tutorial we will a subset of the the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)
* The pre-trained model will be hosted on another worker.

## Setup

### Download/install needed repos
* Clone the [PyGrid repo](https://github.com/OpenMined/PyGrid)
  * we need this because *alice* and *bob* are two different Nodes in our network
  * install the PyGrid node component using *poetry*

### Bring up the PyGridNodes
* In the *PyGrid* repo:
 1. install *poetry* (```pip install poetry```)
 2. go to *apps/nodes*
 3. run ```poetry install``` (those steps are also in the README from the PyGrid repo)
 4. start *bob* and *alice* using:
 ```
 ./run.sh --id alice --port 3000 --start_local_db
 ./run.sh --id bob --port 3001 --start_local_db
 ```
 
This will start two workers, *alice* and *bob* and we will connect to them using the port 3000 and 3001.

In [1]:
!wget "https://raw.githubusercontent.com/facebookresearch/CrypTen/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py" -O "mnist_utils.py"
!wget "https://github.com/facebookresearch/CrypTen/blob/master/tutorials/models/tutorial4_alice_model.pth?raw=true" -O "alice_pretrained_model.pth"
!python ./mnist_utils.py --option train_v_test

--2020-08-06 00:04:44--  https://raw.githubusercontent.com/facebookresearch/CrypTen/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7401 (7.2K) [text/plain]
Saving to: ‘mnist_utils.py’

mnist_utils.py      100%[===================>]   7.23K  --.-KB/s    in 0.005s  

2020-08-06 00:04:44 (1.32 MB/s) - ‘mnist_utils.py’ saved [7401/7401]

--2020-08-06 00:04:44--  https://github.com/facebookresearch/CrypTen/blob/master/tutorials/models/tutorial4_alice_model.pth?raw=true
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/facebookresearch/CrypTen/raw/master/tutorials/models/tutorial4_alice_mod

## Prepare the ground

In [2]:
import crypten

import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time

import syft as sy

from syft.frameworks.crypten.model import OnnxModel
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient
from syft.grid.clients.model_centric_fl_client import ModelCentricFLClient
from syft.frameworks.crypten.context import run_multiworkers

torch.manual_seed(0)
torch.set_num_threads(1)
hook = sy.TorchHook(torch)

## Neural network that will be known only to the workers

*Alice* has the pre-trained version of the model.

In [3]:
class AliceNet(nn.Module):
    def __init__(self):
        super(AliceNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
 
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        return out

## Setup the workers and send them the neural network

We need to have the two GridNodes workers running.

In our current scenario we are sending the serialized model to the workers that are taking part in the computation, but in a real life situation this sending part should not exist - we only need to know that we have the same model on all the workers.

### Scenario
* The local worker wants to run inference on the data that is hosted on *bob* machine.
* The model structure is known only by *alice* and *bob*
* *Alice* has the pre-trained network

In [4]:
# Syft workers
print("[%] Connecting to workers...")
alice = DataCentricFLClient(hook, "ws://localhost:3000")
bob = DataCentricFLClient(hook, "ws://localhost:3001")
print("[+] Connected to workers")

print("[%] Create the serialized model...")
dummy_input = torch.empty((1, 784))
pytorch_model = AliceNet()

# Alice has the model with the real weights
print("[%] Sending the serialized pre-trained model...")
model_pretrained = torch.load('alice_pretrained_model.pth')
model_alice = OnnxModel.fromModel(model_pretrained, dummy_input).tag("crypten_model")
alice_model_ptr = model_alice.send(alice)
print("[+] Serialized model sent to Alice")
    
print("[%] Sending the serialized model...")
model = OnnxModel.fromModel(pytorch_model, dummy_input).tag("crypten_model")
bob_model_ptr = model.send(bob)
print("[+] Serialized model sent to Bob")
    
print("[%] Send test data to bob...")
data = torch.load('/tmp/bob_test.pth')
data_ptr_bob = data.tag("crypten_data").send(bob)
print("[+] Data sent to bob")

print("[%] Load labels...")
labels = torch.load('/tmp/bob_test_labels.pth').long()
print("[+] Labels loaded")


# Function used to compute the accuracy for the model
# Taken from CrypTen repository
def compute_accuracy(output, labels):
    pred = output.argmax(1)
    correct = pred.eq(labels)
    correct_count = correct.sum(0, keepdim=True).float()
    accuracy = correct_count.mul_(100.0 / output.size(0))
    return accuracy

[%] Connecting to workers...
[+] Connected to workers
[%] Create the serialized model...
[%] Sending the serialized pre-trained model...
[+] Serialized model sent to Alice
[%] Sending the serialized model...
[+] Serialized model sent to Bob
[%] Send test data to bob...
[+] Data sent to bob
[%] Load labels...
[+] Labels loaded


### Define the CrypTen computation

We need to specify for the ```run_multiworkers``` decorater:
* the workers that will take part in the computation
* the master address, this will be used for communication

We will use the ```func2plan``` decorator to:
* trace the operations from our function
* sending the plan operations to *alice* and *bob* - the plans operations will act as the function
* run the plans operations on both workers

In [5]:
ALICE = 0 # Alice rank in CrypTen
BOB = 1 # Bob rank in CrypTen

@run_multiworkers([alice, bob], master_addr="127.0.0.1")
@sy.func2plan()
def run_encrypted_inference(crypten=crypten):
    data_enc = crypten.load("crypten_data", BOB)
    
    data_enc2 = data_enc[:100]
    data_flatten = data_enc2.flatten(start_dim=1)
    
    # This should load the crypten model that is found at all parties
    model = crypten.load_model("crypten_model")

    model.encrypt(src=ALICE)
    model.eval()
    
    result_enc = model(data_flatten)
    result = result_enc.get_plain_text()
    
    return result

## Run the CrypTen computation

Now let's run the inference.

In [6]:
# Get the returned values
# key 0 - return values for alice
# key 1 - return values for bob
print("[%] Starting computation")
result = run_encrypted_inference()[1]
print("[+] Computation finished")

accuracy = compute_accuracy(result, labels[:100])
print(f"The accuracy is {accuracy.item()}")

[%] Starting computation


[+] Computation finished
The accuracy is 99.0


## CleanUp

In [8]:
# CleanUp portion taken from the CrypTen project

import os


filenames = ['/tmp/alice_train.pth', 
             '/tmp/alice_train_labels.pth', 
             '/tmp/bob_test.pth', 
             '/tmp/bob_test_labels.pth',
             'alice_pretrained_model.pth',
             'mnist_utils.py']

for fn in filenames:
    if os.path.exists(fn): os.remove(fn)

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22Good+first+issue+%3Amortar_board%3A%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)